In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### Functions

In [3]:
def dist(stop_lat, stop_lon,next_o_lat,next_o_lon):
    return np.arccos(
        np.sin(np.radians(stop_lat)) * np.sin(np.radians(next_o_lat)) + 
        np.cos(np.radians(stop_lat)) * np.cos(np.radians(next_o_lat)) * 
            np.cos(np.radians(stop_lon) - np.radians(next_o_lon))
    ) * 6371

#### Read Origin/Next-Origin Pairs

In [4]:
trips_origins = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_user_trips.csv', parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])

In [5]:
trips_origins.sort_values(['cardNum','o_boarding_id']).head(20)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,next_o_boarding_id,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff
32608,233641.0,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,0 days 05:28:56.000000000
32609,233641.0,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,NaN
93114,257342.0,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,0 days 09:35:53.000000000
93115,257342.0,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,NaN
134081,304127.0,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,0 days 00:32:46.000000000
134082,304127.0,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,NaN
165949,304627.0,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,0 days 02:07:17.000000000
165950,304627.0,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,NaN
138994,305601.0,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,0 days 02:25:53.000000000
138995,305601.0,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,NaN


In [6]:
trips_origins['dist_between_origins'] = trips_origins.apply(lambda x: dist(x['o_stop_lat'],x['o_stop_lon'],x['next_o_stop_lat'],x['next_o_stop_lon']), axis=1)
trips_origins['boardings_timediff'] = pd.to_timedelta(trips_origins['boardings_timediff'])

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [7]:
trips_origins['boardings_timediff'] = np.where(np.isnat(trips_origins['boardings_timediff']),
                                             trips_origins['o_boarding_datetime'] - trips_origins['next_o_boarding_datetime'],
                                             trips_origins['boardings_timediff'])

In [8]:
trips_origins.sort_values(['cardNum','o_boarding_id']).head(50)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
32608,233641.0,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,...,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,05:28:56,14.636225
32609,233641.0,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,...,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,05:28:56,14.636225
93114,257342.0,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,...,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,09:35:53,3.178960
93115,257342.0,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,...,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,09:35:53,3.178960
134081,304127.0,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,...,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,00:32:46,2.639323
134082,304127.0,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,...,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,00:32:46,2.639323
165949,304627.0,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,...,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,02:07:17,8.248431
165950,304627.0,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,...,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,02:07:17,8.248431
138994,305601.0,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,...,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,02:25:53,6.307331
138995,305601.0,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,...,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,02:25:53,6.307331


In [9]:
trips_origins.dist_between_origins.describe()

count    176588.000000
mean          5.467415
std           4.161731
min           0.000000
25%           2.052165
50%           5.142366
75%           8.200742
max          28.419669
Name: dist_between_origins, dtype: float64

In [10]:
trips_origins.boardings_timediff.describe()

count                    177756
mean     0 days 05:49:04.257307
std      0 days 04:05:58.037363
min             0 days 00:00:01
25%      0 days 01:38:57.750000
50%             0 days 05:49:27
75%             0 days 09:49:12
max             0 days 16:26:35
Name: boardings_timediff, dtype: object

In [11]:
len(trips_origins)

177756

In [12]:
trips_origins_clean = trips_origins[(trips_origins['dist_between_origins'] > 1.5) &
                                    (trips_origins['boardings_timediff'] > pd.Timedelta('5 min'))]

In [13]:
len(trips_origins_clean)

139498

In [14]:
trips_origins.dtypes

cardNum                             float64
o_boarding_id                         int64
o_boarding_datetime          datetime64[ns]
o_route                              object
o_busCode                            object
o_tripNum                           float64
o_stopPointId                         int64
o_gps_datetime               datetime64[ns]
o_stop_lat                          float64
o_stop_lon                          float64
next_o_boarding_id                    int64
next_o_boarding_datetime     datetime64[ns]
next_o_route                         object
next_o_busCode                       object
next_o_tripNum                      float64
next_o_stopPointId                    int64
next_o_gps_datetime          datetime64[ns]
next_o_stop_lat                     float64
next_o_stop_lon                     float64
boardings_timediff          timedelta64[ns]
dist_between_origins                float64
dtype: object

#### Read OTP Suggestions

In [156]:
otp_suggestions = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/otp-itineraries/2017_05_09_otp_itineraries.csv', parse_dates=['date','otp_start_time','otp_end_time'])
otp_suggestions['otp_start_time'] = otp_suggestions['otp_start_time'] - pd.Timedelta('10800 s')

In [157]:
otp_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,2017-05-09,1496,1,1,2017-05-09 05:14:11,2017-05-09 08:17:49,WALK,NaN,NaN,NaN,3.633333
1,2017-05-09,1496,1,2,2017-05-09 05:17:50,2017-05-09 08:45:00,BUS,272.0,27222.0,26365.0,27.166667
2,2017-05-09,1496,1,3,2017-05-09 05:45:00,2017-05-09 08:47:44,WALK,NaN,NaN,NaN,2.733333
3,2017-05-09,1496,1,4,2017-05-09 05:55:00,2017-05-09 09:15:56,BUS,304.0,25891.0,26286.0,20.933333
4,2017-05-09,1496,1,5,2017-05-09 06:15:56,2017-05-09 09:16:35,WALK,NaN,NaN,NaN,0.650000


In [158]:
selected_trips = trips_origins_clean[trips_origins_clean['o_boarding_id'].isin(otp_suggestions['user_trip_id'])]

In [159]:
len(selected_trips)

920

In [160]:
selected_trips.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,10:04:22,6.653455
1,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,10:04:22,6.653455
2,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,10:52:10,10.974491
3,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,10:52:10,10.974491
4,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,10:22:43,14.935055


In [161]:
selected_suggestions = otp_suggestions.query('user_trip_id == 247072')

In [162]:
selected_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
39946,2017-05-09,247072,1,1,2017-05-09 14:50:11,2017-05-09 17:54:00,BUS,335.0,33577.0,30027.0,3.816667
39947,2017-05-09,247072,1,2,2017-05-09 14:54:00,2017-05-09 17:56:08,WALK,NaN,NaN,NaN,2.133333
39948,2017-05-09,247072,1,3,2017-05-09 14:57:00,2017-05-09 18:19:59,BUS,535.0,30030.0,36879.0,22.983333
39949,2017-05-09,247072,1,4,2017-05-09 15:20:00,2017-05-09 18:35:14,WALK,NaN,NaN,NaN,15.233333
39950,2017-05-09,247072,2,1,2017-05-09 14:45:57,2017-05-09 17:58:59,WALK,NaN,NaN,NaN,13.033333


In [163]:
itineraries_start = otp_suggestions.query('mode == \'BUS\'') \
                    .groupby(['user_trip_id','itinerary_id']) \
                    .first() \
                    .reset_index() \
                    .assign(route = lambda x: x['route'].astype(int).astype(str))

In [164]:
itineraries_start.head()

,user_trip_id,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,1496,1,2017-05-09,2,2017-05-09 05:17:50,2017-05-09 08:45:00,BUS,272,27222.0,26365.0,27.166667
1,1496,2,2017-05-09,1,2017-05-09 05:03:32,2017-05-09 08:20:00,BUS,226,34486.0,30452.0,16.466667
2,1496,3,2017-05-09,2,2017-05-09 05:23:03,2017-05-09 08:55:00,BUS,270,31703.0,26358.0,31.950000
3,1496,4,2017-05-09,1,2017-05-09 05:28:22,2017-05-09 08:44:00,BUS,226,34486.0,30452.0,15.633333
4,1496,5,2017-05-09,2,2017-05-09 05:40:25,2017-05-09 09:11:00,BUS,271,31703.0,26359.0,30.583333


In [169]:
selected_trips.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_stop_id'], how='inner') \
                .drop_duplicates(subset=['cardNum','o_boarding_id'])

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09,1,2017-05-09 04:41:40,2017-05-09 07:54:01,BUS,548,31593.0,31572.0,12.350000
7,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,1,2017-05-09,1,2017-05-09 14:50:11,2017-05-09 17:54:00,BUS,335,33577.0,30027.0,3.816667
10,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2,2017-05-09,1,2017-05-09 04:57:38,2017-05-09 08:18:00,BUS,548,44395.0,4661.0,20.366667
16,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2,2017-05-09,1,2017-05-09 04:58:28,2017-05-09 08:18:00,BUS,548,31597.0,4661.0,19.533333
18,1232958.0,14667,2017-05-09 04:45:04,548,GA205,1.0,48762,2017-05-09 04:44:52,-25.563193,-49.253670,...,2,2017-05-09,1,2017-05-09 04:46:13,2017-05-09 08:06:00,BUS,548,48762.0,4661.0,19.783333
26,715315.0,2901,2017-05-09 04:45:14,684,HA602,1.0,39368,2017-05-09 04:43:41,-25.588450,-49.341190,...,1,2017-05-09,1,2017-05-09 04:43:58,2017-05-09 08:18:00,BUS,684,39368.0,31053.0,34.033333
32,1982410.0,44254,2017-05-09 04:45:28,684,HA602,1.0,39369,2017-05-09 04:45:27,-25.592110,-49.342120,...,1,2017-05-09,1,2017-05-09 04:45:02,2017-05-09 08:15:45,BUS,684,39369.0,34160.0,30.716667
39,3786597.0,255466,2017-05-09 04:45:34,684,HA602,1.0,39369,2017-05-09 04:45:27,-25.592110,-49.342120,...,1,2017-05-09,1,2017-05-09 04:45:02,2017-05-09 08:18:00,BUS,684,39369.0,31053.0,32.966667
45,985954.0,8715,2017-05-09 04:45:39,542,GA124,1.0,31252,2017-05-09 04:45:23,-25.545960,-49.269090,...,1,2017-05-09,1,2017-05-09 04:46:22,2017-05-09 08:03:08,BUS,542,31252.0,29978.0,16.766667
46,985954.0,8716,2017-05-09 16:40:32,822,BA012,10.0,33801,2017-05-09 15:40:20,-25.437812,-49.363118,...,1,2017-05-09,1,2017-05-09 16:42:00,2017-05-09 19:56:00,BUS,822,33801.0,30994.0,14.000000


In [170]:
terminal_selected_trips = selected_trips[selected_trips['o_busCode'].str.isdigit()]

In [171]:
terminal_selected_trips.merge(itineraries_start, left_on=['o_boarding_id'], right_on=['user_trip_id'], how='inner') \
                .drop_duplicates(subset=['cardNum','o_boarding_id'])

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,1,2017-05-09,2,2017-05-09 15:41:31,2017-05-09 19:06:00,BUS,603,25398.0,26296.0,24.483333
10,3609013.0,201241,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,...,1,2017-05-09,2,2017-05-09 15:14:20,2017-05-09 18:21:00,BUS,303,25757.0,25555.0,6.666667
20,1940885.0,42211,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,...,1,2017-05-09,1,2017-05-09 16:13:00,2017-05-09 19:33:37,BUS,617,31051.0,38521.0,20.616667
30,1232958.0,14668,2017-05-09 15:04:54,000,00024,NaN,43347,NaT,-25.475898,-49.249983,...,1,2017-05-09,2,2017-05-09 15:06:42,2017-05-09 18:27:39,BUS,594,31227.0,31450.0,20.950000
40,715315.0,2902,2017-05-09 13:33:44,000,09070,NaN,43723,NaT,-25.433715,-49.270258,...,1,2017-05-09,2,2017-05-09 13:37:00,2017-05-09 17:07:00,BUS,550,25906.0,26298.0,30.000000
50,1982410.0,44255,2017-05-09 18:38:20,000,00070,NaN,14487,NaT,-25.481927,-49.246999,...,1,2017-05-09,2,2017-05-09 18:40:00,2017-05-09 21:45:41,BUS,602,25474.0,25463.0,5.683333
60,3786597.0,255468,2017-05-09 14:52:19,000,05034,NaN,26083,NaT,-25.426983,-49.264765,...,1,2017-05-09,2,2017-05-09 15:01:00,2017-05-09 18:40:00,BUS,204,25918.0,26252.0,39.000000
70,2178773.0,54397,2017-05-09 05:57:54,000,03044,NaN,41826,NaT,-25.439459,-49.271565,...,1,2017-05-09,2,2017-05-09 06:08:33,2017-05-09 09:12:33,BUS,303,25554.0,25552.0,4.000000
79,2178773.0,54398,2017-05-09 15:41:51,000,06057,NaN,26011,NaT,-25.435634,-49.256611,...,1,2017-05-09,2,2017-05-09 15:48:26,2017-05-09 18:55:00,BUS,303,25646.0,26185.0,6.566667
89,3532686.0,185983,2017-05-09 14:59:59,000,01028,NaN,26033,NaT,-25.453390,-49.252651,...,1,2017-05-09,2,2017-05-09 15:16:53,2017-05-09 18:37:00,BUS,550,25807.0,26298.0,20.116667


#### Read Bus Trips data

In [166]:
bus_trips = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_bus_trips.csv', parse_dates=['gps_datetime'])

In [148]:
bus_trips.head()

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station
0,654,HA017,2.0,35350,2017-05-09 06:28:49,-25.492665,-49.303977,NaN
1,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,NaN
2,654,HA240,3.0,36094,2017-05-09 06:55:54,-25.501283,-49.320023,NaN
3,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,NaN
4,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,NaN


In [161]:
len(bus_trips)

606996

In [149]:
bus_trips.dtypes

route                     object
busCode                   object
tripNum                  float64
stopPointId                int64
gps_datetime      datetime64[ns]
stop_lat                 float64
stop_lon                 float64
parent_station           float64
dtype: object

#### Find trips whose destination is reacheable from origin by taking the boarding bus

In [164]:
od_trips = bus_trips.merge(trips_origins_clean, left_on=['route','busCode','tripNum'], 
                           right_on=['o_route','o_busCode','o_tripNum'], how='inner') \
                    .sort_values(['cardNum','o_boarding_datetime','gps_datetime']) \
                    .query('gps_datetime > o_boarding_datetime')

In [184]:
len(od_trips)

50987

In [183]:
len(od_trips.drop_duplicates(subset=['cardNum','o_route','o_busCode','o_tripNum','o_boarding_datetime']))

1854

In [166]:
od_trips[['cardNum','o_route','o_busCode','o_tripNum','route','busCode','tripNum','stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime']] \
        .sort_values(['cardNum','o_boarding_datetime','gps_datetime']).head(50)

,cardNum,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime
85040,331166.0,965,MN606,15.0,965,MN606,15.0,33371,2017-05-09 20:07:29,2017-05-09 20:08:00,2017-05-09 07:53:13
85041,331166.0,965,MN606,15.0,965,MN606,15.0,33306,2017-05-09 20:07:29,2017-05-09 20:09:10,2017-05-09 07:53:13
85042,331166.0,965,MN606,15.0,965,MN606,15.0,33376,2017-05-09 20:07:29,2017-05-09 20:09:34,2017-05-09 07:53:13
85043,331166.0,965,MN606,15.0,965,MN606,15.0,33382,2017-05-09 20:07:29,2017-05-09 20:10:42,2017-05-09 07:53:13
85044,331166.0,965,MN606,15.0,965,MN606,15.0,26346,2017-05-09 20:07:29,2017-05-09 20:11:35,2017-05-09 07:53:13
85045,331166.0,965,MN606,15.0,965,MN606,15.0,33390,2017-05-09 20:07:29,2017-05-09 20:12:13,2017-05-09 07:53:13
85046,331166.0,965,MN606,15.0,965,MN606,15.0,33310,2017-05-09 20:07:29,2017-05-09 20:13:46,2017-05-09 07:53:13
85047,331166.0,965,MN606,15.0,965,MN606,15.0,33450,2017-05-09 20:07:29,2017-05-09 20:14:52,2017-05-09 07:53:13
85048,331166.0,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13
85049,331166.0,965,MN606,15.0,965,MN606,15.0,26448,2017-05-09 20:07:29,2017-05-09 20:16:06,2017-05-09 07:53:13


In [167]:
od_trips

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station,cardNum,o_boarding_id,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
85040,965,MN606,15.0,33371,2017-05-09 20:08:00,-25.400821,-49.312561,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85041,965,MN606,15.0,33306,2017-05-09 20:09:10,-25.404896,-49.313743,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85042,965,MN606,15.0,33376,2017-05-09 20:09:34,-25.407791,-49.313072,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85043,965,MN606,15.0,33382,2017-05-09 20:10:42,-25.407703,-49.309823,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85044,965,MN606,15.0,26346,2017-05-09 20:11:35,-25.429344,-49.271138,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85045,965,MN606,15.0,33390,2017-05-09 20:12:13,-25.410597,-49.307468,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85046,965,MN606,15.0,33310,2017-05-09 20:13:46,-25.412069,-49.303053,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85047,965,MN606,15.0,33450,2017-05-09 20:14:52,-25.413864,-49.298043,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85048,965,MN606,15.0,26900,2017-05-09 20:15:37,-25.422483,-49.284328,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228
85049,965,MN606,15.0,26448,2017-05-09 20:16:06,-25.411622,-49.294794,NaN,331166.0,160,...,2017-05-09 07:53:13,011,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228


In [168]:
od_trips['dist'] = od_trips.apply(lambda x: dist(x['stop_lat'],x['stop_lon'],x['next_o_stop_lat'],x['next_o_stop_lon']), axis=1)

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [169]:
od_trips[['cardNum','o_route','o_busCode','o_tripNum','route','busCode','tripNum','stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime','dist']].head(30)

,cardNum,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime,dist
85040,331166.0,965,MN606,15.0,965,MN606,15.0,33371,2017-05-09 20:07:29,2017-05-09 20:08:00,2017-05-09 07:53:13,3.314109
85041,331166.0,965,MN606,15.0,965,MN606,15.0,33306,2017-05-09 20:07:29,2017-05-09 20:09:10,2017-05-09 07:53:13,3.090123
85042,331166.0,965,MN606,15.0,965,MN606,15.0,33376,2017-05-09 20:07:29,2017-05-09 20:09:34,2017-05-09 07:53:13,2.840697
85043,331166.0,965,MN606,15.0,965,MN606,15.0,33382,2017-05-09 20:07:29,2017-05-09 20:10:42,2017-05-09 07:53:13,2.587805
85044,331166.0,965,MN606,15.0,965,MN606,15.0,26346,2017-05-09 20:07:29,2017-05-09 20:11:35,2017-05-09 07:53:13,2.038588
85045,331166.0,965,MN606,15.0,965,MN606,15.0,33390,2017-05-09 20:07:29,2017-05-09 20:12:13,2017-05-09 07:53:13,2.202349
85046,331166.0,965,MN606,15.0,965,MN606,15.0,33310,2017-05-09 20:07:29,2017-05-09 20:13:46,2017-05-09 07:53:13,1.755588
85047,331166.0,965,MN606,15.0,965,MN606,15.0,33450,2017-05-09 20:07:29,2017-05-09 20:14:52,2017-05-09 07:53:13,1.260855
85048,331166.0,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13,0.569992
85049,331166.0,965,MN606,15.0,965,MN606,15.0,26448,2017-05-09 20:07:29,2017-05-09 20:16:06,2017-05-09 07:53:13,1.309328


In [185]:
od_trips_dest = od_trips.query('dist <= 1.0') \
                        .groupby(['cardNum','o_boarding_datetime','o_route','o_busCode','o_tripNum']) \
                        .first().reset_index() \
                        .sort_values(['cardNum','o_boarding_datetime']) \
                        .assign(trip_duration = lambda x: x['gps_datetime'] - x['o_boarding_datetime'])

In [186]:
od_trips_dest.head()

,cardNum,o_boarding_datetime,o_route,o_busCode,o_tripNum,route,busCode,tripNum,stopPointId,gps_datetime,...,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins,dist,trip_duration
0,331166.0,2017-05-09 20:07:29,965,MN606,15.0,965,MN606,15.0,26900,2017-05-09 20:15:37,...,JB302,2.0,32796,2017-05-09 07:52:52,-25.422572,-49.290002,12:14:16,3.634228,0.569992,00:08:08
1,352491.0,2017-05-09 17:32:45,901,MC303,11.0,901,MC303,11.0,33658,2017-05-09 17:48:36,...,MC300,3.0,32369,2017-05-09 07:17:11,-25.402222,-49.327749,10:13:52,5.600896,0.811769,00:15:51
2,384263.0,2017-05-09 11:25:04,916,MA299,3.0,916,MA299,3.0,33341,2017-05-09 12:12:32,...,07001,NaN,14510,NaT,-25.400933,-49.330279,01:20:18,3.093219,0.316732,00:47:28
3,405013.0,2017-05-09 07:27:40,911,MN400,3.0,911,MN400,3.0,34350,2017-05-09 07:33:27,...,07902,NaN,14510,NaT,-25.400933,-49.330279,05:39:03,2.694138,0.910208,00:05:47
4,411687.0,2017-05-09 09:31:13,901,MC303,3.0,901,MC303,3.0,32796,2017-05-09 09:38:08,...,07020,NaN,14432,NaT,-25.422391,-49.291635,07:21:57,2.095410,0.165152,00:06:55


In [187]:
od_trips_dest[['cardNum','o_route','o_busCode','o_tripNum','o_stopPointId','o_boarding_datetime','gps_datetime','next_o_boarding_datetime','dist','trip_duration','boardings_timediff']] \
        .sort_values(['cardNum','o_boarding_datetime'])

,cardNum,o_route,o_busCode,o_tripNum,o_stopPointId,o_boarding_datetime,gps_datetime,next_o_boarding_datetime,dist,trip_duration,boardings_timediff
0,331166.0,965,MN606,15.0,32360,2017-05-09 20:07:29,2017-05-09 20:15:37,2017-05-09 07:53:13,0.569992,00:08:08,12:14:16
1,352491.0,901,MC303,11.0,26722,2017-05-09 17:32:45,2017-05-09 17:48:36,2017-05-09 07:18:53,0.811769,00:15:51,10:13:52
2,384263.0,916,MA299,3.0,33447,2017-05-09 11:25:04,2017-05-09 12:12:32,2017-05-09 12:45:22,0.316732,00:47:28,01:20:18
3,405013.0,911,MN400,3.0,34372,2017-05-09 07:27:40,2017-05-09 07:33:27,2017-05-09 13:06:43,0.910208,00:05:47,05:39:03
4,411687.0,901,MC303,3.0,27256,2017-05-09 09:31:13,2017-05-09 09:38:08,2017-05-09 16:53:10,0.165152,00:06:55,07:21:57
5,445228.0,965,MN606,10.0,33376,2017-05-09 17:10:38,2017-05-09 17:44:13,2017-05-09 06:36:47,0.418205,00:33:35,10:33:51
6,643000.0,967,MN600,3.0,33439,2017-05-09 07:15:07,2017-05-09 07:38:39,2017-05-09 18:32:57,0.668432,00:23:32,11:17:50
7,643000.0,967,MN604,13.0,26345,2017-05-09 18:32:57,2017-05-09 18:54:11,2017-05-09 07:15:07,0.640512,00:21:14,11:17:50
8,652419.0,901,MC491,16.0,26455,2017-05-09 15:06:07,2017-05-09 15:16:34,2017-05-09 13:17:31,0.998556,00:10:27,01:48:36
9,669176.0,967,MN604,2.0,26345,2017-05-09 07:27:21,2017-05-09 07:42:25,2017-05-09 19:13:03,0.999929,00:15:04,11:45:42


In [188]:
len(od_trips_dest)

861

In [189]:
od_trips_dest.trip_duration.describe()

count                       861
mean     0 days 00:14:33.634146
std      0 days 00:16:58.419148
min             0 days 00:00:04
25%             0 days 00:06:45
50%             0 days 00:12:36
75%             0 days 00:19:20
max             0 days 07:06:04
Name: trip_duration, dtype: object

In [181]:
od_trips_dest.dist.describe()

count    861.000000
mean       0.246562
std        0.269727
min        0.000000
25%        0.019221
50%        0.164025
75%        0.381333
max        0.984331
Name: dist, dtype: float64

In [159]:
od_trips_dest.cardNum.value_counts()

3187918.0     2
3250023.0     1
2722488.0     1
1654074.0     1
3788749.0     1
2958180.0     1
771608.0      1
3064671.0     1
1565224.0     1
3498692.0     1
3503554.0     1
384263.0      1
3591477.0     1
2652722.0     1
3484961.0     1
2791181.0     1
2410247.0     1
3804412.0     1
2185602.0     1
3469816.0     1
2054995.0     1
10088111.0    1
3462829.0     1
3636750.0     1
2053596.0     1
3276476.0     1
2704065.0     1
3131077.0     1
3439575.0     1
10077853.0    1
3773414.0     1
2282732.0     1
2346224.0     1
2342388.0     1
3778305.0     1
Name: cardNum, dtype: int64

In [140]:
len(od_trips)

109318

In [27]:
candidate_vehicles = enhanced_buste[(enhanced_buste['parent_station'] == enhanced_buste['stopPointId'].iloc[8]) & 
                               ((pd.to_datetime(enhanced_buste['gps_datetime']) > (pd.to_datetime(enhanced_buste['boarding_datetime'].iloc[8]) - pd.Timedelta(value=5,unit='m'))) & 
                                (pd.to_datetime(enhanced_buste['gps_datetime']) < (pd.to_datetime(enhanced_buste['boarding_datetime'].iloc[8]) + pd.Timedelta(value=20,unit='m'))))]

In [28]:
candidate_vehicles

,cardNum,boarding_datetime,route,busCode,tripNum,gps_datetime,stopPointId,stop_lat,stop_lon,parent_station,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,problem,lineName,birthdate,gender
112368,NaN,NaT,203,BD138,1.0,2017-05-01 09:16:17,25732,-25.448323,-49.287777,26105.0,3855.0,4649347.0,-25.448370,-49.287847,5065.989,NO_PROBLEM,NaN,NaN,NaN
114415,NaN,NaT,203,BD147,2.0,2017-05-01 09:26:48,25732,-25.448323,-49.287777,26105.0,3855.0,4649347.0,-25.448370,-49.287847,5065.989,NO_PROBLEM,NaN,NaN,NaN
232947,NaN,NaT,603,HD249,3.0,2017-05-01 09:14:53,25732,-25.448323,-49.287777,26105.0,3156.0,3758012.0,-25.448258,-49.287815,7432.708,NO_PROBLEM,NaN,NaN,NaN
233467,NaN,NaT,603,HD250,2.0,2017-05-01 09:33:05,25732,-25.448323,-49.287777,26105.0,3156.0,3758012.0,-25.448258,-49.287815,7432.708,BETWEEN,NaN,NaN,NaN


In [29]:
def find_candidate_vehicles(row,gps_df,lower_time_thresh = 0, higher_time_thresh = 40):
    return gps_df[(gps_df['parent_station'] == row['stopPointId']) & 
                               ((pd.to_datetime(gps_df['gps_datetime']) > (pd.to_datetime(row['boarding_datetime']) - pd.Timedelta(value=lower_time_thresh,unit='m'))) & 
                                (pd.to_datetime(gps_df['gps_datetime']) < (pd.to_datetime(row['boarding_datetime']) + pd.Timedelta(value=higher_time_thresh,unit='m'))))]

In [30]:
first_boarding = 40
second_boarding = 41

In [31]:
test_data = enhanced_buste.loc[first_boarding:second_boarding].head(1)

In [32]:
test_data

,cardNum,boarding_datetime,route,busCode,tripNum,gps_datetime,stopPointId,stop_lat,stop_lon,parent_station,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,problem,lineName,birthdate,gender
40,341036.0,2017-05-01 08:57:07,021,08025,NaN,NaT,41751,-25.441058,-49.3425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTERB II ANTI H,09/07/69,F


In [33]:
nstop_lat = enhanced_buste['stop_lat'].loc[second_boarding]
nstop_lon = enhanced_buste['stop_lon'].loc[second_boarding]

In [34]:
nstop_lat, nstop_lon

(-25.4376552204, -49.266631957100003)

In [36]:
dist(-7.213345, -35.907678, -7.219092, -35.877251)

3.4168198120895932

In [37]:
def is_vehicle_suitable(row,gps_df,next_stop_lat,next_stop_lon,higher_time_thresh=2):
    vehicle_trip = gps_df[(gps_df['busCode'] == row['busCode']) & (gps_df['tripNum'] == row['tripNum'])].sort_values('gps_datetime')
    vehicle_suitability = vehicle_trip[(vehicle_trip['gps_datetime'] > row['gps_datetime'])].apply(lambda row: dist(row['stop_lat'],row['stop_lon'],next_stop_lat,next_stop_lon), axis=1) \
                            .reset_index(name='dist')
    #print vehicle_suitability['dist']
    return len(vehicle_suitability[vehicle_suitability['dist'] <= 1.0]) > 0
    

In [38]:
for index, row in test_data.iterrows():
    cand_vehicles = find_candidate_vehicles(row,enhanced_buste)
    suitable_candidates = []
    for cand_index, cand_row in cand_vehicles.iterrows():
        if (is_vehicle_suitable(cand_row,enhanced_buste,nstop_lat, nstop_lon)):
            suitable_candidates.append(cand_row)
    suitable_candidates_df = pd.DataFrame.from_records(suitable_candidates, columns=cand_vehicles.columns)
    print suitable_candidates_df
    chosen_candidate = suitable_candidates_df.loc[suitable_candidates_df['gps_datetime'].idxmin()]
    print chosen_candidate

   cardNum boarding_datetime route busCode  tripNum        gps_datetime  \
0      NaN               NaT   303   DE710      2.0 2017-05-01 09:31:02   
1      NaN               NaT   303   DE713      2.0 2017-05-01 09:17:39   
2      NaN               NaT   303   DE719      2.0 2017-05-01 09:05:31   

   stopPointId   stop_lat   stop_lon  parent_station  shapeId  shapeSequence  \
0        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   
1        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   
2        25748 -25.441061 -49.342507         41751.0   3617.0      4399768.0   

    shapeLat   shapeLon  distanceTraveledShape     problem  lineName  \
0 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   
1 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   
2 -25.441017 -49.342554                456.152  NO_PROBLEM       NaN   

   birthdate  gender  
0        NaN     NaN  
1        NaN     NaN  
2        NaN     NaN